# Μεταφορά Μάθησης (Transfer Learning)

Σε αυτό το notebook, θα μάθετε πώς να χρησιμοποιείτε προ-εκπαιδευμένα δίκτυα για την επίλυση απαιτητικών προβλημάτων όρασης υπολογιστή. Συγκεκριμένα, θα χρησιμοποιείτε δίκτυα εκπαιδευμένα στο dataset [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

Το ImageNet είναι ένα τεράστιο σύνολο δεδομένων με πάνω από 1 εκατομμύριο εικόνες απο 1000 κατηγορίες. Χρησιμοποιείται για να εκπαιδεύσει βαθιά νευρωνικά δίκτυα χρησιμοποιώντας μια αρχιτεκτονική που ονομάζεται συνελικτικά στρώματα. Δεν πρόκειται να μπούμε σε αυτο το notebook στις λεπτομέρειες των συνελικτικών δικτύων καθώς θα τα αναλύσουμε σε επόμενο μάθημα.

Μόλις εκπαιδευτούν, αυτά τα μοντέλα λειτουργούν εκπληκτικά, ως ανιχνευτές χαρακτηριστικών (feature detectors) για εικόνες στις οποίες δεν εκπαιδεύτηκαν. Η χρήση ενός προ-εκπαιδευμένου δικτύου σε εικόνες που δεν εχουν συμπεριληθεί στο σύνολο των εικόνων εκπαίδευσης ονομάζεται μεταφορά μάθησης. Εδώ θα χρησιμοποιήσουμε την μεταφορά μάθησης για να εκπαιδεύσουμε ένα δίκτυο που θα μπορεί να ταξινομεί φωτογραφίες γάτας και σκύλου με σχεδόν τέλεια ακρίβεια.

Μέσω του `torchvision.models` μπορείτε να κατεβάσετε αυτά τα προ-εκπαιδευμένα δίκτυα και να τα χρησιμοποιήσετε στις εφαρμογές σας. Θα συμπεριλάβουμε πλέον και τα `models` στα imports μας.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Τα περισσότερα από τα προ-εκπαιδευμένα μοντέλα απαιτούν την είσοδο εικόνων με ανάλυση 224x224. Επίσης, θα πρέπει να αντιστοιχίσουμε την κανονικοποίηση που χρησιμοποιήθηκε κατά την εκπαίδευση αυτών των μοντέλων. Κάθε κανάλι χρώματος κανονικοποιήθηκε ξεχωριστά, με τις μέσες τιμές να είναι `[0,448, 0,456, 0,406]` και οι τυπικές αποκλίσεις να είναι `[0.229, 0.224, 0.225]`.

In [ ]:
data_dir = 'Cat_Dog_data'

# Συμπληρώστε: Ορίστε τους μετασχηματισμούς για τα δεδομένα εκπαίδευσης και τα δεδομένα ελέγχου
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Περάστε τους μετασχηματισμούς εδώ και, στη συνέχεια, εκτελέστε το επόμενο κελί για να δείτε πώς φαίνονται οι μετασχηματισμοί
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

Μπορούμε να φορτώσουμε ένα μοντέλο όπως το [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Ας εμφανίσουμε την αρχιτεκτονική του μοντέλου για να δούμε τι συμβαίνει.

In [ ]:
model = models.densenet121(pretrained=True)
model

Αυτό το μοντέλο είναι κατασκευασμένο από δύο κύρια μέρη, τα χαρακτηριστικά και τον ταξινομητή. Το τμήμα χαρακτηριστικών είναι μια στοίβα συνελικτικών επιπέδων και λειτουργεί συνολικά ως ανιχνευτής χαρακτηριστικών που μπορεί να τροφοδοτηθεί σε έναν ταξινομητή. Το τμήμα του ταξινομητή είναι ένα ενιαίο πλήρως συνδεδεμένο επίπεδο `(classifier): Linear (in_features = 1024, out_features = 1000)`. Αυτό το επίπεδο εκπαιδεύτηκε στο σύνολο δεδομένων ImageNet, οπότε δεν θα λειτουργήσει για το συγκεκριμένο πρόβλημα. Αυτό σημαίνει ότι πρέπει να αντικαταστήσουμε τον ταξινομητή, αλλά τα χαρακτηριστικά θα λειτουργούν τέλεια από μόνα τους. Σε γενικές γραμμές, θεωρώ τα προ-εκπαιδευμένα δίκτυα ως εκπληκτικά καλούς ανιχνευτές χαρακτηριστικών που μπορούν να χρησιμοποιησω ως είσοδο σε απλούς ταξινομητές.

In [ ]:
# "Παγώνω" τις παραμέτρους, ώστε να μην αλλάζουν κατα το backpropagation
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
model

Με δομημένο πλέον το μοντέλο μας, πρέπει να εκπαιδεύσουμε τον ταξινομητή. Ωστόσο, τώρα χρησιμοποιούμε ένα **πραγματικά βαθύ** νευρωνικό δίκτυο. Εάν προσπαθήσετε να το εκπαιδεύσετε σε μια CPU όπως συνήθως, θα χρειαστεί πολύ πολύ χρόνος. Αντ 'αυτού, θα χρησιμοποιήσουμε την GPU για να κάνουμε τους υπολογισμούς. Οι γραμμικοί υπολογισμοί άλγεβρας γίνονται παράλληλα στην GPU που οδηγεί σε 100νταπλάσιες ταχύτητες εκπαίδευσης. Είναι επίσης δυνατό να εκπαιδεύσετε το δίκτυο σε πολλές GPU, μειώνοντας περαιτέρω τον χρόνο εκπαίδευσης.

Η PyTorch, όπως επίσης και κάθε άλλη library/framework βαθιάς μάθησης, χρησιμοποιεί [CUDA] (https://developer.nvidia.com/cuda-zone) για να υπολογίζει αποτελεσματικά τα βήματα πρόσω και πίσω τροφοδότησης στην GPU. Στη PyTorch, μπορούμε να μεταφέρουμε τις παραμέτρους του μοντέλου μας και άλλους τανυστές στη μνήμη GPU χρησιμοποιώντας την εντολή `model.to('cuda')`. Μπορείτε να τα μεταφέρετε πίσω στην CPU με την `model.to('cpu')` εντολή που θα εκτελείτε συνήθως όταν θα πρέπει να χρησιμοποιήσετε την έξοδο του δικτύου σας εκτός PyTorch. Ως απόδειξη της αυξημένης ταχύτητας, θα συγκρίνω πόσο χρόνο χρειάζεται για να εκτελέσουμε ένα βήμα προς τα εμπρός και πίσω με και χωρίς GPU.

In [ ]:
import time

In [ ]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Εκπαιδεύστε μόνο τις παραμέτρους ταξινομητή, οι παράμετροι των χαρακτηριστικών έχουν παγώσει
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Μετακινήστε την είσοδο και τις ετικέτες στην GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Μπορείτε να γράψετε device agnostic code που θα χρησιμοποιεί αυτόματα τη CUDA εάν είναι ενεργοποιημένη οπως παρακάτω:

```python
# στην αρχή του κώδικα
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# έπειτα όποτε παίρνετε ένα νέο τανυστή ή Module 
# δεν θα τα αντιγράφει εάν βρίσκονται ήδη στον επιθυμητό επεξεργαστή
input = data.to(device)
model = MyModule(...).to(device)
```

Από εδώ, θα σας αφήσω να ολοκληρώσετε την εκπαίδευση του μοντέλου. Η διαδικασία είναι η ίδια όπως πριν, εκτός του ότι τώρα το μοντέλο σας είναι πολύ πιο ισχυρό. Θα πρέπει να επιτύχετε εύκολα ακρίβεια μεγαλύτερη από 95%.

>**Άσκηση:** Εκπαιδεύστε προ-εκαπιδευμένα μοντέλα για να ταξινομήσετε εικόνες γάτας και σκύλου. Συνεχίστε με το μοντέλο DenseNet ή δοκιμάστε το ResNet, το οποίο είναι επίσης ένα καλό μοντέλο για να δοκιμάσετε. Βεβαιωθείτε ότι εκπαιδεύετε μόνο τον ταξινομητή και ότι οι παράμετροι για το τμήμα των χαρακτηριστικών έχουν παγώσει.

In [ ]:
# Αν δεν υπάρχει GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# "Παγώστε" τις παραμέτρους, ώστε να μην αλλάζουν κατα το backpropagation
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Εκπαιδεύστε μόνο τις παραμέτρους του ταξινομητή, οι παράμετροι των χαρακτηριστικών έχουν παγώσει
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Μετακινήστε τις εισόδου και τις ετικέτες στον προεπιλεγμένο επεξεργαστή
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Υπολογίστε την ακρίβεια
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Εποχή {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>